In [33]:
import whisper
import os
import pandas as pd
import subprocess
from tqdm.notebook import tqdm

pd.set_option('display.max_colwidth', None)

In [ ]:
# Create wav directory
os.makedirs("GV_Eval_3h/wav", exist_ok=True)

# Convert mp3 files to wav format
for file in os.listdir("GV_Eval_3h/Audio"):
    if file.endswith(".mp3"):
        input_file = os.path.join("GV_Eval_3h/Audio", file)
        output_file = os.path.join("GV_Eval_3h/wav", f"{os.path.splitext(file)[0]}.wav")
        command = ["ffmpeg", "-i", input_file, "-ar", "16000", "-ac", "1", output_file]
        subprocess.run(command)

In [11]:
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [06:54<00:00, 7.46MiB/s]


In [13]:
from IPython.display import Audio, display
WAV_FILE_PATH = 'D:\Coding\Projects\Kathbath_WER\GV_Eval_3h\wav'
display(Audio(f'{WAV_FILE_PATH}\\01-00004-02.wav', rate = 32_000,autoplay=True))

In [16]:
# let's transcribe the file that we just listened  above
mp3_file = f'{WAV_FILE_PATH}\\01-00004-02.wav'
model.transcribe(mp3_file,language='hi',fp16=False)['text']

' वहीं राहुल कुमार सहीय कुमार कुमार महिक कुमार सेव जर्ज खरक ग्रतल ही सामन से कहिंदjach दिंह कंडे के मोघर और नृधव इधमान यूजर से'

In [17]:
texts =  pd.read_csv('D:\Coding\Projects\Kathbath_WER\GV_Eval_3h\\text',delimiter = '\t',header = None)

In [18]:
# helpers
def file_name(string:str)->str:
    return string.split()[0]
def transcript(string:str)->str:
    return ' '.join(string.split()[1:])

In [19]:
meta_df = pd.DataFrame()
meta_df['file_name'] = texts.head(100)[0].apply(file_name)
meta_df['original_translation'] = texts.head(100)[0].apply(transcript)
meta_df

,file_name,original_translation
0,01-00004-02,वहीं राहुल कुमार मुकेश कुमार अरुण कुमार राजा कुमार अनुराग कुमार सनी कुमार रहीश कुमार मनीष कुमार सहित दर्जनों कार्यकर्ता भी शामिल थे धन्यवाद दिलीप पांडे जमुई अघहारा से बिहार मोबइल वाणी न्यूज़ एक्सप्रेस से
1,01-00008-03,युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या पैसे खर्च करने की ज़रुरत नहीं है दिन
2,01-00031-03,क्यूँकि हमारी पुलिस तो फेसबुक और दुसरे सोशल मीडिया के माध्यमों पर अपराध करने वालों को पकड़ने में ही असमर्थ है
3,01-00071-02,नाली जाम है जिससे निकालता गन्दा पानी सड़को पर जमा है जिसकी सफाई को लेकर किसी प्रकार की कयावत होती भी नहीं दिखती है
4,01-00078-01,समस्या को सुना जायेगा और महिलाओं की समस्या महिलाओं समझ सकती है इस इस विचार को रखते हुए सरकार ने महिलाओं के प्र
...,...,...
95,01-01992-01,करोड़ों रुपया में मिला हुआ है वैसे भी किसी बच्ची के नाम पर दस हजार बीस हजार पच्चीस हजार तीस हजार रुपया तक बाकया हो जाता है
96,01-01998-03,मैं फैजान बोल रहा हूँ रामगढ़ जिले से हमारे यहाँ कोयले की बहुत सी खदानें खोदी गयी हैं और उन खदानों मैं चलने वाली गाड़ियाँ
97,01-02007-03,हम सभी सहमत हो गए कि गरीब किसी जात या समुदाय का छात्र हो हम सभी भाई भाई हैं फिर भेदभाव क्यूँ हम लोग एक साथ खाना प्रारंभ किया
98,01-02017-03,मैं टीएन ब्रह्मर्षि शांति क्लब की ओर से बिहार मोबाइल वाणी के लिए मधुबनी से आज हम बाबूबरही प्रखंड का दौरा किया और मध्य विद्यालय


In [20]:
import re

def remove_ids_from_file(file_path):
    # Read the contents of the file
    with open(file_path, 'r', encoding='utf-8') as file:
        transcriptions = file.readlines()

    # Remove IDs from each transcription
    cleaned_transcriptions = []
    for transcription in transcriptions:
        cleaned_transcription = re.sub(r'^\d{2}-\d{5}-\d{2}\s+', '', transcription)
        cleaned_transcriptions.append(cleaned_transcription.strip())

    # Write the cleaned transcriptions back to the file
    with open('reference.txt', 'w', encoding='utf-8') as file:
        for cleaned_transcription in cleaned_transcriptions:
            file.write(cleaned_transcription + '\n')

remove_ids_from_file("D:\Coding\Projects\Kathbath_WER\GV_Eval_3h\\text")

In [22]:
ref = open('reference.txt').read()
print(ref)

वहीं राहुल कुमार मुकेश कुमार अरुण कुमार राजा कुमार अनुराग कुमार सनी कुमार रहीश कुमार मनीष कुमार सहित दर्जनों कार्यकर्ता भी शामिल थे धन्यवाद दिलीप पांडे जमुई अघहारा से बिहार मोबइल वाणी न्यूज़ एक्सप्रेस से
युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या पैसे खर्च करने की ज़रुरत नहीं है  दिन
क्यूँकि हमारी पुलिस तो फेसबुक और दुसरे सोशल मीडिया के माध्यमों पर अपराध करने वालों को पकड़ने में ही असमर्थ है
नाली जाम है जिससे निकालता गन्दा पानी सड़को पर जमा है जिसकी सफाई को लेकर किसी प्रकार की कयावत होती भी नहीं दिखती है
समस्या को सुना जायेगा और महिलाओं की समस्या महिलाओं समझ सकती है इस इस विचार को रखते हुए सरकार ने महिलाओं के प्र
विकास के लिए शराब बंदी एक बड़ा कदम है इस मुद्दे पर जदूय जि ज़िला अध्यक्ष शंकर चौधरी जदूय नेता
इस मध्य प्रदेश मोबाइल वाणी के सभी सुनने वाले श्रोताओं को और मेरे मित्रों को सभी को हार्दिक हार्दिक शुभकामनाएं धन्यवाद नमस्कार
स्तर गिरने का मुख्य कारण  संस्कृति और परम्पराओं का सबक सबक बच्चों को न पढ़ाना है दुर्भाग्य की बात है कि भारत मैं आज लोग शिक्षा को मात्र डिग्री पाना समझ कर
पुरुष के 

In [23]:
options = dict(language='hi')
transcribe_options = dict(task="transcribe", **options)

In [25]:
# write transcription for first some files and store it in a text file
whisper_transcription_file = 'whisper_transcription.txt'
for audio in tqdm(meta_df['file_name'][:100]):
    audio_file = f'{WAV_FILE_PATH}/{audio}.wav'
    transcription = model.transcribe(audio_file, **transcribe_options ,fp16=False)["text"]
    with open(whisper_transcription_file,'a') as hyp:
        hyp.write(transcription + '\n')
        
        
print(open(whisper_transcription_file).read())

  0%|          | 0/100 [00:00<?, ?it/s]

 वहीं राहुल कुमार वुकेश कुमार राजा कुमार अनुरख कुमार संयिक कुमार रहित कुमार मधिस कुमार शाहिद गर्जेनों कार्यक्राइता भी सामन चेजने वक्त दिलपांडे के मौधराशे बिहार मुझ्वानी लिजल प्रेस्टूर्श
 यह बहुत ख्याल जिमेदारी है इसलिए बहुत समय या भी से कर करने की ज़रूरत नहीं है कोई विश्वपिन नहीं है
 क्योंकि हमारी पुलिस का फेस्बुक और गुशे सुसल मिग्रे के माध्यमों पर अप्राज करने वालों को पकड़ने में ही असमर्थ है
 नाले जान है जिससे निकला दंदा पाने सडको पजमा है, सिस्टा पाइल लेकर किसी प्रकार की कयावत होती भी नहीं कितितिया है।
 समस्या को सुना जाएगा और महलाओं की समस्या महलाओं समझ सकती है इस विचार को रखते हुए सरकार में महलाओं के पर
 विकास के लिए सरावन दी एक बड़ा कदम है इस मोटे पर जिर्वियों जिला देश संकत सोधरी जरीव नेचा
 इस मंद प्रदेश मुझे बाद के सली सुनने वाले सुरुताओं को और मेरे मित्रों को तभी को हार्षिक हार्षिक सुप्ताओं ले धन्यवाद नवस्कार
 प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश्न प्रश

In [27]:
# from the master reference file, just get the references for the audio files that we have transcripted above
original_transcription_file = 'original_transcription.txt'
for transcript in tqdm(meta_df['original_translation'][:100]):
    with open(original_transcription_file,'a') as ref:
        ref.write(transcript + '\n')
        
print(open(original_transcription_file).read())

  0%|          | 0/100 [00:00<?, ?it/s]

वहीं राहुल कुमार मुकेश कुमार अरुण कुमार राजा कुमार अनुराग कुमार सनी कुमार रहीश कुमार मनीष कुमार सहित दर्जनों कार्यकर्ता भी शामिल थे धन्यवाद दिलीप पांडे जमुई अघहारा से बिहार मोबइल वाणी न्यूज़ एक्सप्रेस से
युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या पैसे खर्च करने की ज़रुरत नहीं है दिन
क्यूँकि हमारी पुलिस तो फेसबुक और दुसरे सोशल मीडिया के माध्यमों पर अपराध करने वालों को पकड़ने में ही असमर्थ है
नाली जाम है जिससे निकालता गन्दा पानी सड़को पर जमा है जिसकी सफाई को लेकर किसी प्रकार की कयावत होती भी नहीं दिखती है
समस्या को सुना जायेगा और महिलाओं की समस्या महिलाओं समझ सकती है इस इस विचार को रखते हुए सरकार ने महिलाओं के प्र
विकास के लिए शराब बंदी एक बड़ा कदम है इस मुद्दे पर जदूय जि ज़िला अध्यक्ष शंकर चौधरी जदूय नेता
इस मध्य प्रदेश मोबाइल वाणी के सभी सुनने वाले श्रोताओं को और मेरे मित्रों को सभी को हार्दिक हार्दिक शुभकामनाएं धन्यवाद नमस्कार
स्तर गिरने का मुख्य कारण संस्कृति और परम्पराओं का सबक सबक बच्चों को न पढ़ाना है दुर्भाग्य की बात है कि भारत मैं आज लोग शिक्षा को मात्र डिग्री पाना समझ कर
पुरुष के प्

In [28]:
# calculate the WER using subprocess to run the command as if on the command line
command = f"wer {original_transcription_file} {whisper_transcription_file}"
process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
print(type(output.decode()))
print(output.decode())

<class 'str'>
Sentence count: 100
WER:    58.014% (      1531 /       2639)
WRR:    47.025% (      1241 /       2639)
SER:   100.000% (       100 /        100)



In [30]:
import torch
import librosa
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor

model_id = "D:\Coding\Projects\\vistaar\hindi_models\whisper-medium-hi_alldata_multigpu"
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def transcribe_audio(audio_filepath):
    audio_input, sampling_rate = librosa.load(audio_filepath, sr=16000)
    
    input_values = processor(audio_input, return_tensors="pt", sampling_rate=16000).input_features.to(device)
    forced_decoder_ids = processor.get_decoder_prompt_ids(language="en", task="transcribe")
    with torch.no_grad():
        predicted_ids = model.generate(input_values, forced_decoder_ids=forced_decoder_ids, return_dict_in_generate=True)
    
    transcription = processor.batch_decode(predicted_ids.sequences, skip_special_tokens=True)[0]
    
    return transcription

# Example usage
audio_filepath = "D:\Coding\Projects\Kathbath_WER\GV_Eval_3h\wav\\01-00004-02.wav"
transcription = transcribe_audio(audio_filepath)
print("Transcription:", transcription)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Transcription: वही राहुल कुमार मुकेश कुमार राजा कुमार अनुराग कुमार संजय कुमार रहीश कुमार मनीष कुमार सहित दर्जनों कार्यकर्ता भी शामिल थे धन्यवाद दिलीप पांडे जमुई अघरा ऐसी बिहार मोबाइल वाणी मीडिया डेस्क ऐसी


In [31]:
# write transcription for first some files and store it in a text file
indic_whisper_transcription_file = 'indic_whisper.txt'
for audio in tqdm(meta_df['file_name'][:100]):
    audio_file = f'D:\Coding\Projects\Kathbath_WER\GV_Eval_3h\wav\\{audio}.wav'
    transcription = transcribe_audio(audio_file)
    with open(indic_whisper_transcription_file, 'a') as indic:
        indic.write(transcription + '\n')
        
        
print(open(indic_whisper_transcription_file).read())

  0%|          | 0/100 [00:00<?, ?it/s]

वही राहुल कुमार मुकेश कुमार राजा कुमार अनुराग कुमार संजय कुमार रहीश कुमार मनीष कुमार सहित दर्जनों कार्यकर्ता भी शामिल थे धन्यवाद दिलीप पांडे जमुई अघरा ऐसी बिहार मोबाइल वाणी मीडिया डेस्क ऐसी
ये बात की आम जिम्मेदारी है इसके लिए बहुत समय या पैसे खर्च करने की जरुरत नहीं है कोई नहीं कुछ दिन
क्यूँकी हमारी पुलिस का फेसबुक और दुसरे सोशल मीडिया के माध्यमों आरोप अपराध करने वालों को पकड़ने में ही असमर्थ है
नाले जम है जिससे निकालना गन्दा पानी सड़कों आरोप जमा है इसकी सफाई लेकर किसी प्रकार की प्रयावरण होती भी नहीं दिख दिखती है
समस्या को सुना जायेगा और महिलाओं की समस्या महिलाओं समझ सकती है इस इस विचार को रखते हुए सरकार ने महिलाओं के प्रति
विकास के लिए शराब बूंदी एक बड़ा कदम है इस मौके आरोप जिला अध्यक्ष चंद्र चौधरी नेता
इस मार्ग प्रदेश मोबाइल वाणी के सभी सुनने वाले श्रोताओं को और मेरे मित्रों को सभी को हार्दिक हार्दिक शुभ कामनायें धन्यवाद नमस्कार
इसी का असर गिरने का मुख्य कारण इस संस्कृति और परम्पराओ का सबक सबक बच्चों को न पढ़ाना है दुर्भाग्य की बात है की भारत में आज लोक शिक्षा को मात्र जिद्दी पाना सम

In [32]:
command = f"wer {original_transcription_file} {indic_whisper_transcription_file}"
process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
print(type(output.decode()))
print(output.decode())

<class 'str'>
Sentence count: 100
WER:    31.982% (       844 /       2639)
WRR:    70.822% (      1869 /       2639)
SER:   100.000% (       100 /        100)

